In [72]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
  $('div.input').hide();
  $('div.output_prompt').css("visibility", "hidden");
 } else {
  $('div.input').show();
  $('div.output_prompt').css("visibility", "visible");
 }
 code_show = !code_show;
} 
$(document).ready(code_toggle);
$(document).dblclick(code_toggle);
</script>''')

# Mortgage scenarios

Housing is nuts. This is a quick (rough) running of the numbers for a number of the options that I am considering, the end goal (at the moment) being able to live/work in a small to medium sized warehouse in West Auckland somewhere in five years time. There are a small number of suitable warehouses going at the moment in the 6-700K range. The problem with these is that banks want a much greater deposit (~40%) and have much shorter terms for loans on commercial buildings.  

There are a number of scenarios to be considered. All should be evaluated against the end goal, which at this point in time is to accumalate sufficeint value so as to be able to purchase a property on my own in 5(?) years time.

Options so far include
- Kiwibuild new build
- Kiwibuild existing house
- New build in Auckland
- New build out of Auckland
- Existing build in Auckland
- Existing build out of Auckland
- Small brick and tile
- Leasehold
- Commercial (warehouse)

Not currently going to evaluate all of these, but enough of them to get an idea of what is viable given different deposit amounts, and then evaulate against the end goal criteria. They are all by neccesity, approximate. Some assumptions used:
- higher than current interest rate, to guard against the fact that the interest rates will go up. 
- capital gains are only assumed on some options
- captial gains are assumed to be lower than they have been for the last few years


In [3]:
# relevant code/imports for all sections. 

from ipywidgets import interact, widgets
from IPython.display import display
from mortgage import Loan
import inspect
from IPython.display import Markdown as md
import pandas as pd
from IPython.display import display, HTML

# for widget description width
style = {'description_width': 'initial'}

def installmentToDict(installment):
    installmentDict=dict()
    for i in inspect.getmembers(installment):
    # to remove private and protected functions
        if not i[0].startswith('_'):
        # To remove other methods that do not start with a underscore
            if not inspect.ismethod(i[1]): 
                installmentDict[i[0]]=i[1]
    del installmentDict["count"]
    del installmentDict["index"]
    
    #for key, value in installmentDict.items():
    #    if isinstance(value, Decimal):
    #        installmentDict[key] = float(value)

    return installmentDict

def compound_gains(principle, rate, time):
    # time is in years
    n = 1 #number of periods at which the interest is accured - 1 is annually.
    amount = principle * (((1 + ((rate/100.0)/n)) ** (n*time)))
    return(amount)

def interestToDate(loanSchedule, currentMark):
    interestPaid = 0
    for i in range(1,currentMark*12):
        values = installmentToDict(loanSchedule.schedule(currentMark))
        interestPaid = interestPaid + values['interest']
    return interestPaid

def principleToDate(loanSchedule, currentMark):
    principalPaid = 0
    for i in range(1,currentMark*12):
        values = installmentToDict(loanSchedule.schedule(currentMark))
        principalPaid = principalPaid + values['principal']
    return principalPaid

def format(x):
        return "${:.1f}K".format(x/1000)
    
## for changing the slider variables
def on_change(v):
    x = v['new'] 


## Kiwibuild, new build

The new limit for first home buyers for new builds utilising Kiwibuild is 700K. There are currently builds available (with unknown start dates) for ~650. The minimum deposit using this approach is 5%. Caveats with kiwibuild
- requires my income as a single person be less than 95K
- requires any loan/gift for a deposit to be declared a gift and not be expected to be repaid. 

In [2]:

loan = Loan(principal=500000, interest=.06, term=30)
loan.summarize

Original Balance:         $    500,000
Interest Rate:                    0.06 %
APY:                              6.17 %
APR:                              6.00 %
Term:                               30 years
Monthly Payment:          $    2997.75

Total principal payments: $ 500,000.00
Total interest payments:  $ 579,190.95
Total payments:           $1,079,190.95
Interest to principal:           115.8 %
Years to pay:                     30.0


## Kiwbuild, existing build
The current limit on using Kiwibuild to but an existing home is $630K. This is not a lot, The benefits are that it would allow me to access about 8K in grants and buy using only 5% deposit

Two options
- one bedroom, only have to live in it for a year
- two bedrooms, have to live in it for 3 years. 



In [284]:
## assumed rate of capital gains, default to the Auckland average of 5%
capitalGainsRate=5.2
## the number of years. 
mark=5
## interest rate, expressed as a value between 0 and 1
interestRate=0.06
## the number of years the loan would run for if I didn't cash out. 
term = 30
## total value of the hypothetical property
totalValue = 750000

## value of deposit from Sam
depositValue = 200000


## sum borrowed from the bank
loanValue = totalValue - depositValue

## minimum house value at the mark, assuming capital gains rate stays at at least the default
finalValue = compound_interest(loanValue, capitalGainsRate, mark)

## loan computation
loan = Loan(principal=loanValue, interest=interestRate, term = term)

valuesAtMark = installmentToDict(loan.schedule(mark*12))
valuesAtMark['interestToDate'] = interestToDate(loan, mark)
valuesAtMark['interestToDate'] = valuesAtMark.pop('total_interest')
valuesAtMark['principleToDate'] = principleToDate(loan, mark)

# capital gains
valuesAtMark['valueAtMark'] = compound_gains(totalValue, capitalGainsRate, mark)
valuesAtMark['capitalGains'] = valuesAtMark['valueAtMark'] - totalValue
valuesAtMark['personalGains'] = valuesAtMark['capitalGains']*(1-(depositValue/totalValue)) + loanValue-float(valuesAtMark['balance'])
valuesAtMark['gainsOnDeposit'] = valuesAtMark['capitalGains']*(depositValue/totalValue)

valuesAtMark['interestOnDeposit'] = (depositValue * interestRate) * mark

valuesAtMark['weeklyMortgagePayment'] = float(loan.monthly_payment) * 12/52
valuesAtMark['weeklyPayment'] = valuesAtMark['weeklyMortgagePayment'] + (depositValue * interestRate)/52

markValues = pd.DataFrame(valuesAtMark.items(), columns =["field", "value"]) 
markValues['value'] = markValues['value'].apply('${:,.2f}'.format)
markValues.iloc[2,1] = valuesAtMark['number']

display(markValues)

a=2


,field,value
0,balance,"$511,798.96"
1,interest,"$2,562.67"
2,number,60
3,payment,"$3,297.53"
4,principal,$734.86
5,interestToDate,"$159,650.64"
6,principleToDate,"$32,955.09"
7,valueAtMark,"$966,362.26"
8,capitalGains,"$216,362.26"
9,personalGains,"$196,866.70"


## New build in Auckland

{{loanValue}}
"This is a string with {{" + a + "}} in it"


## New build, out of Auckland

## Existing build, in Auckland

## Existing build, out of Auckland

## Small brick and tile.


## Leasehold


Leasehold is a little different. The upfront sum is significantly less. There is a significant cost involved in the ground rent though. The upfront sum could be anywhere from 150 to 250k. On the face of it, it would be very easy to pay this off in a few years. Ground rent can easily add 300 pw to the costs though, making repayments smaller. Things to be wary about - when is the ground rent due for review, CVu had a thing last year where an extra 10K/year was added to the ground rent. A review just gone might make this an attractive option for a couple of years, or a review several years away, giving time to buy, recoup and sell with a signiicant period left before the next review - will be harder to sell if the next review is close. 

In [86]:



propertyValueInput=widgets.IntText(value=200, description= "Total Value(k): ", style = style)
display(propertyValueInput)

depositInput=widgets.IntText(value=40, description= "Deposit(k): " , style = style)
display(depositInput)

groundRentInput=widgets.IntText(value=12, description= "Ground rent(k): ", style = style)
display(groundRentInput)

x=3
interestSlider = widgets.FloatSlider(value=3, min=0, max=10, description="Interest rate(%)" , readout_format='.2f', style = style)
interestSlider.observe(on_change, names='value')
display(interestSlider)


termInput=widgets.IntText(value=10, description= "Term(y): ", style = style)
display(termInput)


IntText(value=200, description='Total Value(k): ', style=DescriptionStyle(description_width='initial'))

IntText(value=40, description='Deposit(k): ', style=DescriptionStyle(description_width='initial'))

IntText(value=12, description='Ground rent(k): ', style=DescriptionStyle(description_width='initial'))

FloatSlider(value=3.0, description='Interest rate(%)', max=10.0, style=SliderStyle(description_width='initial'…

IntText(value=10, description='Term(y): ', style=DescriptionStyle(description_width='initial'))

If we add a proviso of limiting the rent to around $700 - flexible, but roughly 500(-100) for me, 200(+/-100) for a flatmate, then paying around with the settings, we can see that with ~300pw added for ground rent, then we have to have a term of ~10 years to make the rent achievable. 

In [91]:
interestRate= interestSlider.value/100
totalValue = propertyValueInput.value * 1000
depositValue=depositInput.value * 100
term = termInput.value
loan = Loan(principal=totalValue-depositValue, interest=interestRate, term=term)


fiveYearMark = installmentToDict(loan.schedule(60))

weeklyMortgage = (fiveYearMark['payment']*12)/52
weeklyGroundRent = groundRentInput.value*1000/52
weeklyMortgage = float(weeklyMortgage)
weeklyPayment = weeklyMortgage + weeklyGroundRent

print("weekly mortgage payment:")
print("${:,.2f}".format(weeklyMortgage))
print("weekly payment including ground rent/opex:")
print("${:,.2f}".format(weeklyPayment))


weekly mortgage payment:
$402.72
weekly payment including ground rent/opex:
$729.65


So a shorter term, with more goes towards the capital. If we look at this at the five year mark then what remains is:

In [94]:
fiveYearMark = installmentToDict(loan.schedule(60))
print ("${:,.2f}".format(totalValue-depositValue-fiveYearMark["balance"]))



$81,140.52


Which might be fine if there were capital gains, but the general advice appears to be that the capital gains accrue to the land, and not the built structures. So total gains on this appear to be ~ 80K over five years. 

Most of these options have a cost - namely, the interest. This option has the added cost of 

In [102]:
groundRentPaid = weeklyGroundRent * 52 * term
print ("${:,.2f}".format(groundRentPaid))

$187,000.00


has been paid to get to this point, a combination of ground rent and opex in a roughly 2/3-1/3 split. 
As an aside, if one was okay with being a landlord, and could afford to buy one of these outright, it might possibly be an okay thing to contemplate, as the only cost is that of your ground rent/opex. 

## Commercial

Commercial properties require a minimum of 40% deposit, which makes them difficult to obtain finance for. Additionally, interest rates are higher, and loan terms shorter. 

Currently, the BNZ appears to be the only viable option as it is possible to get a 25 year term loan, as opposed to the standard 15 that other banks appear to have as their limit. 


Interest rates for the BNZ vary, with a base of 2.2% and an industry specific rate that can vary between 1.8 and 4. 

In [103]:

propertyValue=widgets.IntText(value=650, description= "Total Value: ")

display(propertyValue)

depositValue=widgets.IntText(value=120, description= "Deposit: ")

display(depositValue)


x = 5
slider = widgets.IntSlider(value=4, min=0, max=10)
# change the slider value
def on_change(v):
    x = v['new'] 
slider.observe(on_change, names='value')

display(slider)

IntText(value=650, description='Total Value: ')

IntText(value=120, description='Deposit: ')

IntSlider(value=4, max=10)

In [104]:


interestRate= slider.value/100

totalValue = propertyValue.value * 1000
depositValue=depositValue.value * 100

loan = Loan(principal=totalValue-depositValue, interest=interestRate, term=25)



At the 5 year mark, the capital that I would hold would be:
Value of property - sams deposit - sams share of capital gains - whatever is left to the bank. 

In [105]:


fiveYearMark = installmentToDict(loan.schedule(60))

for key, value in fiveYearMark.items():
    if key=="number":
        print(key, " : ", value)
    else:
        print(key, " : ", "${:,.2f}".format(value))


balance  :  $555,727.45
interest  :  $1,857.46
number  :  60
payment  :  $3,367.60
principal  :  $1,510.14
total_interest  :  $119,783.39


So my capital becomes is what remains when the deposit and what the bank still owns is removed form the total value:

In [106]:

print ("${:,.2f}".format(totalValue-depositValue-fiveYearMark["balance"]))

$82,272.55


This is without accounting for capital gains. Auckland has been at ~5% for some time, so I'm currently seeting capital gains at 4%, which leaves a total value of:

In [107]:
gainsRate=5
mark=5
  
finalValue = compound_interest(totalValue, gainsRate, mark)
print("${:,.2f}".format(finalValue))



$829,583.02


And a final total capital for myself of:

In [108]:
personalGains = float((finalValue-totalValue)/2)
fiveYearMark["balance"] = float(fiveYearMark["balance"])
      
print ("${:,.2f}".format(totalValue + personalGains - depositValue - fiveYearMark["balance"]))

$172,064.06


Then the question remains, how much has this cost me? The two sums I am interested in here are how much have I spent, and how much would I have spent on rent/saved otherwise. 
Costs at this point will have been the interest on the loan, plus the interest on the deposit. 
Interest on the loan to this point:

In [114]:
interestPaid = 0
for i in range(1, 5*12):
    interestPaid = interestPaid + loan.schedule(i).interest
    
print("${:,.2f}".format(interestPaid))

$117,925.94


whilst what I would have paid in rent (300pw) would have been:

In [121]:
rent = 200 * 52 * 5
print(rent)

52000


and I would have been able to save maybe ...

In [122]:
savings = 350 * 52 * 5
print(savings)

91000
